## В этом ноутбуке:
1. Работаем с датасетом Fashion Mnist
2. Сделаем и обучим модель, которая будет классифицировать объекты из этого датасета. Всего классов 10
3. Посмотрим на более сложную модель, но дающую лучший результат на валидационной выборке

In [0]:
# Необходимые импорты
from keras import Sequential
import keras
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Reshape, InputLayer, Dropout
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Датасет встроен в Keras, так что его можно легко скачать:

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

В датасете всего 10 классов, перечислим их в списке fashion_mnist_labels:

In [0]:
# Define the text labels
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9


Также посмотрим на изображения:

In [0]:
# Печатаем случайно выбранные 15 изображений:
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(x_test[index]))
    true_index = y_test[index]
    ax.set_title("{}".format(fashion_mnist_labels[true_index]))

Посмотрим каково число объектов в выборках:

In [0]:
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

# Число объектов в обучающей/тестовой выборках:
print(x_train.shape[0], 'train set')
print(x_test.shape[0], 'test set')

Посмотрим как выглядит y_train/y_test

In [0]:
y_train

In [0]:
y_test

__Вопрос__: Задача классификации, значит что необходимо сделать с y_train/y_test?

In [0]:
# <Your code>

Теперь объявим модель. Можете использовать любые слои и всё, что мы узнали за прошедшую неделю

Модель должна достичь примерно 0.83 accuracy на валидачионной выборке

In [0]:
# <Your code>

In [0]:
# Скомпилировать модель:
# <Your code>

In [0]:
# Сделать fit. 10% выборки отвести под валидационную
# <Your code>

In [0]:
# Оценить качество модели на тестовом наборе:
# <Your code>

Теперь сделаем небольшую визуализацию того, как работает модель:

In [0]:
y_hat = model.predict(x_test)

# Plot a random sample of 10 test images, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(x_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(x_test[index]))
    predict_index = np.argmax(y_hat[index])
    true_index = np.argmax(y_test[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index], 
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

Посмотрим теперь только на неверно предсказанные объекты:

In [0]:
y_hat_argmax = np.argmax(y_hat, axis=1)
y_test_argmax = np.argmax(y_test, axis=1)
choose_wrong = ~(y_hat_argmax == y_test_argmax)
y_hat_wrong = y_hat[choose_wrong]
y_test_wrong = y_test[choose_wrong]
x_test_wrong = x_test[choose_wrong]

In [0]:
# Plot a random sample of 10 test images, their predicted labels and ground truth
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(y_hat_wrong.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(x_test_wrong[index]))
    predict_index = np.argmax(y_hat_wrong[index])
    true_index = np.argmax(y_test_wrong[index])
    # Set the title for each image
    ax.set_title("{} ({})".format(fashion_mnist_labels[predict_index], 
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

Наконец, соберем метрики по каждому отдельному классу. Для этого есть метод classification_report в sklearn:

In [0]:
from sklearn.metrics import classification_report
y_test_arg = np.argmax(y_test, axis=1)
y_hat_arg = np.argmax(y_hat, axis=1)

print(classification_report(y_test_arg, y_hat_arg, target_names=fashion_mnist_labels))

In [0]:
# Другая модель, которая работает несколько лучше:

In [0]:
from keras.layers import Activation, BatchNormalization, MaxPooling2D
from keras.optimizers import SGD
chanDim = -1
classes = 10
INIT_LR = 1e-2

model = Sequential()
model.add(InputLayer((28, 28)))
model.add(Reshape((28, 28, -1)))

# first CONV => RELU => CONV => RELU => POOL layer set
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# second CONV => RELU => CONV => RELU => POOL layer set
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# first (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# softmax classifier
model.add(Dense(classes))
model.add(Activation("softmax"))

# return the constructed network architecture

In [0]:
#opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / 30)
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [0]:
!pip install keras-tqdm

In [0]:
from keras_tqdm import TQDMNotebookCallback 

model.fit(x=x_train, y=y_train, epochs=30, batch_size=32, verbose=0,
          validation_split=0.1, callbacks=[TQDMNotebookCallback(leave_inner=True)])

Как видно, модель примерно на 5-ой эпохе достигает качества 0.92. А к 10-ой эпохе ~0.93

__Вопрос:__ Почему на некоторых эпохах accuracy < val_accuracy ?